# Handling Both Sync and Async Tasks with OmniQ

This notebook demonstrates how OmniQ workers can seamlessly handle both synchronous and asynchronous tasks, showcasing the library's flexibility in mixed workload scenarios.

## Key Concepts

- **AsyncWorker**: Executes async tasks natively, runs sync tasks in thread pools
- **ThreadWorker**: Executes sync tasks natively, runs async tasks in new event loops
- **Mixed Workloads**: OmniQ automatically detects and handles both task types appropriately

## Setup and Imports

In [ ]:
import asyncio
import time
from omniq import OmniQ
from omniq.queue import FileTaskQueue
from omniq.workers import AsyncWorker, ThreadWorker

print("OmniQ imported successfully!")

## Define Task Functions

Let's define both synchronous and asynchronous task functions to demonstrate mixed workload handling.

In [ ]:
def sync_task(x, y, operation="multiply"):
    """A synchronous task that performs mathematical operations"""
    print(f"🔄 Executing sync task: {x} {operation} {y}")
    time.sleep(0.1)  # Simulate some CPU work
    
    if operation == "multiply":
        result = x * y
    elif operation == "add":
        result = x + y
    elif operation == "subtract":
        result = x - y
    else:
        result = x / y if y != 0 else 0
    
    print(f"✅ Sync task result: {result}")
    return result


async def async_task(x, y, operation="add"):
    """An asynchronous task that performs mathematical operations with I/O simulation"""
    print(f"🔄 Executing async task: {x} {operation} {y}")
    await asyncio.sleep(0.1)  # Simulate async I/O operation
    
    if operation == "add":
        result = x + y
    elif operation == "multiply":
        result = x * y
    elif operation == "subtract":
        result = x - y
    else:
        result = x / y if y != 0 else 0
    
    print(f"✅ Async task result: {result}")
    return result

print("Task functions defined!")

## Example 1: AsyncWorker with Mixed Tasks

Let's see how an AsyncWorker handles both synchronous and asynchronous tasks.

In [ ]:
print("=== AsyncWorker with Mixed Tasks ===")

# Create components
queue = FileTaskQueue(project_name="notebook_async_example", base_dir="./temp_storage")
worker = AsyncWorker(queue=queue, max_workers=5)

print("📦 Created FileTaskQueue and AsyncWorker")

# Start the worker
worker.start()
print("🚀 AsyncWorker started")

# Enqueue mixed tasks
print("\n📝 Enqueuing tasks...")
sync_task_id = queue.enqueue(sync_task, func_args=dict(x=5, y=10, operation="multiply"))
async_task_id = queue.enqueue(async_task, func_args=dict(x=5, y=10, operation="add"))

print(f"   - Sync task ID: {sync_task_id}")
print(f"   - Async task ID: {async_task_id}")

# Wait for execution
print("\n⏳ Waiting for tasks to execute...")
time.sleep(2)

# Get results
print("\n📊 Retrieving results...")
sync_result = worker.get_result(sync_task_id)
async_result = worker.get_result(async_task_id)

print(f"   - Sync task result: {sync_result}")
print(f"   - Async task result: {async_result}")

# Stop worker
worker.stop()
print("\n🛑 AsyncWorker stopped")

## Example 2: ThreadWorker with Mixed Tasks

Now let's see how a ThreadWorker handles the same mixed workload.

In [ ]:
print("=== ThreadWorker with Mixed Tasks ===")

# Create components with ThreadWorker
queue = FileTaskQueue(project_name="notebook_thread_example", base_dir="./temp_storage")
worker = ThreadWorker(queue=queue, max_workers=5)

print("📦 Created FileTaskQueue and ThreadWorker")

# Start the worker
worker.start()
print("🚀 ThreadWorker started")

# Enqueue mixed tasks
print("\n📝 Enqueuing tasks...")
sync_task_id = queue.enqueue(sync_task, func_args=dict(x=3, y=7, operation="multiply"))
async_task_id = queue.enqueue(async_task, func_args=dict(x=3, y=7, operation="add"))

print(f"   - Sync task ID: {sync_task_id}")
print(f"   - Async task ID: {async_task_id}")

# Wait for execution
print("\n⏳ Waiting for tasks to execute...")
time.sleep(2)

# Get results
print("\n📊 Retrieving results...")
sync_result = worker.get_result(sync_task_id)
async_result = worker.get_result(async_task_id)

print(f"   - Sync task result: {sync_result}")
print(f"   - Async task result: {async_result}")

# Stop worker
worker.stop()
print("\n🛑 ThreadWorker stopped")

## Example 3: Batch Processing with Mixed Tasks

Let's process multiple sync and async tasks together to see how workers handle concurrent mixed workloads.

In [ ]:
print("=== Batch Processing with Mixed Tasks ===")

# Use context managers for automatic cleanup
with FileTaskQueue(project_name="notebook_batch_example", base_dir="./temp_storage") as queue, \
     AsyncWorker(queue=queue, max_workers=10) as worker:
    
    print("📦 Created components with context managers")
    
    # Enqueue multiple mixed tasks
    print("\n📝 Enqueuing batch of mixed tasks...")
    task_ids = []
    operations = ["add", "multiply", "subtract"]
    
    for i in range(3):
        # Sync task
        sync_id = queue.enqueue(
            sync_task, 
            func_args=dict(x=i+1, y=i+2, operation=operations[i])
        )
        
        # Async task
        async_id = queue.enqueue(
            async_task, 
            func_args=dict(x=i+1, y=i+2, operation=operations[i])
        )
        
        task_ids.extend([sync_id, async_id])
        print(f"   - Batch {i+1}: Sync ID {sync_id}, Async ID {async_id}")
    
    # Wait for all tasks to complete
    print("\n⏳ Waiting for all tasks to complete...")
    time.sleep(3)
    
    # Collect all results
    print("\n📊 Collecting all results...")
    results = []
    for i, task_id in enumerate(task_ids):
        result = worker.get_result(task_id)
        task_type = "Sync" if i % 2 == 0 else "Async"
        results.append(result)
        print(f"   - {task_type} task {task_id}: {result}")
    
    print(f"\n✅ Successfully processed {len(task_ids)} mixed tasks!")
    print(f"📈 Results: {results}")

## Example 4: Performance Comparison

Let's compare how different worker types perform with mixed workloads.

In [ ]:
import time

def benchmark_worker(worker_class, worker_name, num_tasks=5):
    """Benchmark a worker with mixed sync/async tasks"""
    print(f"\n🏃 Benchmarking {worker_name}...")
    
    start_time = time.time()
    
    with FileTaskQueue(project_name=f"benchmark_{worker_name.lower()}", base_dir="./temp_storage") as queue, \
         worker_class(queue=queue, max_workers=10) as worker:
        
        # Enqueue mixed tasks
        task_ids = []
        for i in range(num_tasks):
            sync_id = queue.enqueue(sync_task, func_args=dict(x=i, y=i+1, operation="multiply"))
            async_id = queue.enqueue(async_task, func_args=dict(x=i, y=i+1, operation="add"))
            task_ids.extend([sync_id, async_id])
        
        # Wait for completion
        time.sleep(2)
        
        # Collect results
        results = [worker.get_result(task_id) for task_id in task_ids]
        
    end_time = time.time()
    duration = end_time - start_time
    
    print(f"   ⏱️  {worker_name} completed {len(task_ids)} tasks in {duration:.2f} seconds")
    print(f"   📊 Results: {results[:6]}{'...' if len(results) > 6 else ''}")
    
    return duration, results

print("=== Performance Comparison ===")

# Benchmark AsyncWorker
async_duration, async_results = benchmark_worker(AsyncWorker, "AsyncWorker")

# Benchmark ThreadWorker
thread_duration, thread_results = benchmark_worker(ThreadWorker, "ThreadWorker")

print(f"\n🏆 Performance Summary:")
print(f"   - AsyncWorker: {async_duration:.2f}s")
print(f"   - ThreadWorker: {thread_duration:.2f}s")

if async_duration < thread_duration:
    print(f"   🥇 AsyncWorker was {((thread_duration - async_duration) / thread_duration * 100):.1f}% faster")
else:
    print(f"   🥇 ThreadWorker was {((async_duration - thread_duration) / async_duration * 100):.1f}% faster")

print("\n✅ Both workers successfully handled mixed sync/async workloads!")

## Example 5: Interactive Task Execution

Try creating your own tasks and see how OmniQ handles them!

In [ ]:
# Define your own custom tasks here!

def custom_sync_task(message, repeat=1):
    """Custom sync task - modify as needed"""
    result = message * repeat
    print(f"🔄 Custom sync task: '{message}' repeated {repeat} times")
    time.sleep(0.05)  # Small delay
    return result

async def custom_async_task(message, delay=0.1):
    """Custom async task - modify as needed"""
    print(f"🔄 Custom async task: processing '{message}' with {delay}s delay")
    await asyncio.sleep(delay)
    return f"Processed: {message.upper()}"

# Test your custom tasks
print("=== Interactive Custom Tasks ===")

with FileTaskQueue(project_name="interactive_example", base_dir="./temp_storage") as queue, \
     AsyncWorker(queue=queue, max_workers=3) as worker:
    
    print("📦 Ready for custom tasks!")
    
    # Enqueue your custom tasks
    sync_id = queue.enqueue(custom_sync_task, func_args=dict(message="Hello", repeat=3))
    async_id = queue.enqueue(custom_async_task, func_args=dict(message="World", delay=0.2))
    
    print(f"📝 Enqueued custom sync task: {sync_id}")
    print(f"📝 Enqueued custom async task: {async_id}")
    
    # Wait and get results
    time.sleep(1)
    
    sync_result = worker.get_result(sync_id)
    async_result = worker.get_result(async_id)
    
    print(f"\n📊 Custom sync result: {sync_result}")
    print(f"📊 Custom async result: {async_result}")
    
    print("\n🎉 Custom tasks completed successfully!")

## Summary

This notebook demonstrated how OmniQ seamlessly handles both synchronous and asynchronous tasks:

### Key Takeaways:

1. **Automatic Detection**: OmniQ automatically detects whether a task is sync or async
2. **Worker Flexibility**: Both AsyncWorker and ThreadWorker can handle mixed workloads
3. **Performance**: Different workers may perform better depending on the workload characteristics
4. **Ease of Use**: No special configuration needed - just enqueue tasks and let OmniQ handle the execution

### Worker Behavior:

- **AsyncWorker**: 
  - ✅ Native async task execution
  - ✅ Sync tasks run in thread pools
  - 🎯 Best for I/O-heavy workloads

- **ThreadWorker**: 
  - ✅ Native sync task execution
  - ✅ Async tasks run in new event loops
  - 🎯 Best for CPU-heavy workloads

### Next Steps:

- Try modifying the task functions to see different behaviors
- Experiment with different worker configurations
- Test with your own real-world sync and async functions
- Explore other OmniQ features like scheduling and result storage